<a href="https://colab.research.google.com/github/fahim-03/NLP_Labs/blob/main/lab1_preprocessing_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
__author__ = "Pranava Madhyastha"
__version__ = "INM434/IN3045 City, University of London, Spring 2026"

## Tokenisation: a brief overview

This is our first notebook for the module and we are going to focus on tokenisation. Please make sure to login to your respective google account to get started.

A tokenizer is used to split an input string into separate tokens or pieces, where each token represents a meaningful element in the input string. This is one of the most important steps in NLP. This step is also called text-preprocessing.

Tokenisation helps create a vocabulary of unique tokens overwhich we will run a variety of NLP algorithms. It mainly helps standardise the input data and help us squeeze in information from the long tail. It, in some cases, helps us to break the input text down into smaller, sometimes linguistically meaningful, units.

In this notebook, we will first build a simple tokeniser which only splits on white space. We will then look at a toy morphological analyser. We will then build a simplified version of subword based tokeniser.

---

How does this notebook work:

  * There will be some cells with "TODO": you will have to fill the code for the placeholder "YOUR CODE HERE".
  * You will also notice "ADVANCED TODO", this is for students who are indeed capable of writing

  * Each cell should take a few seconds to run, so if it is taking longer, there may be bug.

# Simple tokeniser

---



We will make use of the built in regular expression library in python to construct this tokeniser. Please refer to Chapter 2 (SLP: Jurafsky and Martin) references on regular expressions.

The tokeniser below will take split input text into separate words based on word boundaries. This is defined using the regular expression pattern "\b\w+\b". The re.findall() function is used to extract all the tokens from the text that match the pattern, and the list of tokens is returned.


In [2]:
import re

def tokenise(text):
    # Define a regular expression pattern to split the text into tokens
    pattern = r'\b\w+\b'

    # Use the re.findall() function to extract all the tokens from the text
    tokens = re.findall(pattern, text)

    # Return the list of tokens
    return tokens

# Sample input for the tokeniser
text = "This is IN 3045/INM 434 Natural Language Procssing. This is some sample text for tokeniser"
tokens = tokenise(text)
print(tokens)

['This', 'is', 'IN', '3045', 'INM', '434', 'Natural', 'Language', 'Procssing', 'This', 'is', 'some', 'sample', 'text', 'for', 'tokeniser']


TODO: What does '\b\w+\b' mean?

It splits a series of alphanumerical characters by whitespace

---
# Building a Sentiment Classifier <a name="classifier"></a>

Let's build a sentiment classifier that predicts whether a movie review is **positive** or **negative** -- this is one of the classic NLP tasks.

## Our Plan is:

1. **Tokenise**: Input the text by tokenising the text
2. **Vectorise**: Convert the elements of the text into a bag-of-words representation
3. **Train**: Learn which words indicate positive/negative sentiment
4. **Predict**: Classify new reviews

In [26]:
import re
import numpy as np
from collections import Counter, defaultdict
import matplotlib.pyplot as plt

# Our training data: movie reviews with sentiment labels
training_data = [
    # Positive reviews
    ("This movie is amazing and wonderful", "positive"),
    ("I loved every minute of this film", "positive"),
    ("Brilliant acting and great story", "positive"),
    ("One of the best movies I have seen", "positive"),
    ("Absolutely fantastic and entertaining", "positive"),
    ("A beautiful and moving experience", "positive"),
    ("Highly recommend this masterpiece", "positive"),
    ("Excellent film with superb performances", "positive"),
    ("I really enjoyed this movie", "positive"),
    ("What a great and fun movie", "positive"),
    ("The movie was not bad", "positive"),
    ("I did not hate it", "positive"), # TEACH: NOT_hate is positive

    # Negative reviews
    ("This movie is terrible and boring", "negative"),
    ("I hated every minute of this film", "negative"),
    ("Awful acting and stupid story", "negative"),
    ("One of the worst movies ever made", "negative"),
    ("Absolutely dreadful waste of time", "negative"),
    ("A painful and tedious experience", "negative"),
    ("Do not waste your money on this", "negative"),
    ("Horrible film with bad performances", "negative"),
    ("I really disliked this movie", "negative"),
    ("This movie is not good", "negative"), # TEACH: NOT_good is negative
    ("I don't like this film", "negative"), # TEACH: NOT_like is negative
]

pos_count = 0
neg_count = 0

for data in training_data:
  if(data[1] == "positive"):
    pos_count += 1
  else:
    neg_count += 1

print(f"Total training examples: {len(training_data)}")
print(f"Total positive reviews: {pos_count}")
print(f"Total negative reviews: {neg_count}")



Total training examples: 23
Total positive reviews: 12
Total negative reviews: 11


TODO: Please write code to list the number of training examples in total, with the number of positive and negative examples.

## Simple Tokenizer

# First, we need to split text into words. Let's start with a simple approach:

In [4]:
def simple_tokenize(text):
    # TODO - write code to convert to lowercase -> just ended up using pythons built-in lower() function
    # Write code to Split on word boundaries, keep only alphanumeric
    pattern = r'\b\w+\b'
    tokens = re.findall(pattern, text.lower())

    return tokens

print(simple_tokenize("This is IN 3045/INM 434 Natural Language Procssing. This is some sample text for tokeniser"))

['this', 'is', 'in', '3045', 'inm', '434', 'natural', 'language', 'procssing', 'this', 'is', 'some', 'sample', 'text', 'for', 'tokeniser']


## Building Vocabulary

We need to know all unique words in our training data -- this is the VOCABULARY of our model

In [6]:
def build_vocabulary(data, tokenize_fn):

    # we will use COUNTER class from python -- https://docs.python.org/3/library/collections.html
    word_counts = Counter()

    for text, label in data:
        tokens = tokenize_fn(text)
        word_counts.update(tokens)

    # TODO: What is counter doing? -> it creates a map/dict of all tokens in a corpus/text, in the format token: Frequency

    # Create word-to-index mapping
    vocab = {word: idx for idx, word in enumerate(sorted(word_counts.keys()))}

    return vocab, word_counts

# Build vocabulary
vocab, word_counts = build_vocabulary(training_data, simple_tokenize)
print(f"Vocabulary size: {len(vocab)}")
print(vocab)
print("Words from Most to Least frequent:")
for token, freq in word_counts.most_common():
  print(f"- {token}: {freq}")


Vocabulary size: 61
{'a': 0, 'absolutely': 1, 'acting': 2, 'amazing': 3, 'and': 4, 'awful': 5, 'bad': 6, 'beautiful': 7, 'best': 8, 'boring': 9, 'brilliant': 10, 'disliked': 11, 'do': 12, 'dreadful': 13, 'enjoyed': 14, 'entertaining': 15, 'ever': 16, 'every': 17, 'excellent': 18, 'experience': 19, 'fantastic': 20, 'film': 21, 'fun': 22, 'great': 23, 'hated': 24, 'have': 25, 'highly': 26, 'horrible': 27, 'i': 28, 'is': 29, 'loved': 30, 'made': 31, 'masterpiece': 32, 'minute': 33, 'money': 34, 'movie': 35, 'movies': 36, 'moving': 37, 'not': 38, 'of': 39, 'on': 40, 'one': 41, 'painful': 42, 'performances': 43, 'really': 44, 'recommend': 45, 'seen': 46, 'story': 47, 'stupid': 48, 'superb': 49, 'tedious': 50, 'terrible': 51, 'the': 52, 'this': 53, 'time': 54, 'waste': 55, 'what': 56, 'with': 57, 'wonderful': 58, 'worst': 59, 'your': 60}
Words from Most to Least frequent:
- this: 8
- and: 8
- movie: 5
- i: 5
- of: 5
- film: 4
- a: 3
- is: 2
- every: 2
- minute: 2
- acting: 2
- great: 2
- sto

TODO: Write code to print the size of the vocabulary and also list the most frequent to least frequent words.

## Bag-of-Words Representation

Convert each document to a vector (in our case an array) of word counts

In [8]:
def text_to_bow(text, vocab, tokenize_fn):
    # Initialize vector of zeros
    bow = np.zeros(len(vocab))

    # Count words
    tokens = tokenize_fn(text)
    for token in tokens:
        if token in vocab:
            bow[vocab[token]] += 1

    return bow


example  = "This movie is "
bow_vector = text_to_bow(example, vocab, simple_tokenize)
print (bow_vector)
print ("All tokens with non-zero entries:")
for token, index in vocab.items():
    if bow_vector[index] > 0:
        print(f"- {token}")

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
All tokens with non-zero entries:
- is
- movie
- this


TODO: Print out the bag of word vector. Also, print out all the "tokens" with non-zero entries.

## Now Let us train a Simple Classifier

We will now use a very simple classifier -- the Naive Bayes classifier which is a simple but effective classifier for text.

The idea: estimate P(word | positive) and P(word | negative), then use Bayes' rule.

Essentially we are seeing If a review is positive, will it have the word 'good'?. Then, we ask, if I indeed see the word 'good', what is the probability the review is positive?. It allows us to update our belief about the review's sentiment as we read each word -- this is a fairly naive way of approaching language -- here we don't care about the rich structure!

In [9]:
class NaiveBayesClassifier:


    def __init__(self, tokenize_fn):
        self.tokenize_fn = tokenize_fn
        self.vocab = None
        self.word_probs = {}  # P(word | class)
        self.class_probs = {}  # P(class)

    def train(self, data):

        # Build vocabulary
        self.vocab, word_counts = build_vocabulary(data, self.tokenize_fn)

        # Count words per class -- TODO: Why do we need this?
        # (ANSWER) This is needed to calculate P(word|class) as we want to know the probabilty of a word being in a class given all words in a class
        class_word_counts = defaultdict(Counter)
        class_doc_counts = Counter()

        # Remember each item in data is a string, label pair
        for text, label in data:
            tokens = self.tokenize_fn(text)
            class_word_counts[label].update(tokens)
            class_doc_counts[label] += 1

        # Calculate probabilities with smoothing -- this is done to prevent division by zero!
        total_docs = len(data)
        vocab_size = len(self.vocab)

        for label in class_doc_counts:
            # P(class)
            self.class_probs[label] = class_doc_counts[label] / total_docs

            # P(word | class) with Laplace smoothing
            total_words = sum(class_word_counts[label].values())
            self.word_probs[label] = {}

            for word in self.vocab:
                count = class_word_counts[label][word]
                # Laplace smoothing -- add 1 to avoid zero probabilities
                self.word_probs[label][word] = (count + 1) / (total_words + vocab_size)

        print(f"Trained on {total_docs} documents")
        print(f"Vocabulary size is: {vocab_size}")
        print(f"Total number of classes: {list(class_doc_counts.keys())}")

    def predict(self, text, return_probs=False):

        tokens = self.tokenize_fn(text)

        # Calculate log probability for each class
        log_probs = {}

        for label in self.class_probs:
            # Start with log P(class)
            log_prob = np.log(self.class_probs[label])

            # Add log P(word | class) for each word
            for token in tokens:
                if token in self.word_probs[label]:
                    log_prob += np.log(self.word_probs[label][token])
                # Ignore unknown words (or could use a small probability)

            log_probs[label] = log_prob

        # Predict class with highest probability
        predicted = max(log_probs, key=log_probs.get)

        if return_probs:
            # Convert to actual probabilities
            max_log = max(log_probs.values())
            probs = {k: np.exp(v - max_log) for k, v in log_probs.items()}
            total = sum(probs.values())
            probs = {k: v/total for k, v in probs.items()}
            return predicted, probs

        return predicted

    def get_important_words(self, n=10):

        important = {}

        for label in self.class_probs:
            other_label = [l for l in self.class_probs if l != label][0]

            ratios = []
            for word in self.vocab:
                ratio = self.word_probs[label][word] / self.word_probs[other_label][word]
                ratios.append((word, ratio))

            ratios.sort(key=lambda x: x[1], reverse=True)
            important[label] = ratios[:n]

        return important

# Let us now train our classifier
classifier = NaiveBayesClassifier(simple_tokenize)
classifier.train(training_data)

Trained on 19 documents
Vocabulary size is: 61
Total number of classes: ['positive', 'negative']


## So what has it learned?

Let us see which words are most indicative of positive vs negative sentiment?

In [10]:
important_words = classifier.get_important_words(10)
#print(important_words)
print("Most important words and their ratios:")
for label, word_ratios in important_words.items():
  print(f"\n--- {label.upper()} ---")
  for word, ratio in word_ratios:
    print(f"- Word: '{word}', Ratio: {ratio:.4f}")

Most important words and their ratios:

--- POSITIVE ---
- Word: 'great', Ratio: 2.9224
- Word: 'amazing', Ratio: 1.9483
- Word: 'beautiful', Ratio: 1.9483
- Word: 'best', Ratio: 1.9483
- Word: 'brilliant', Ratio: 1.9483
- Word: 'enjoyed', Ratio: 1.9483
- Word: 'entertaining', Ratio: 1.9483
- Word: 'excellent', Ratio: 1.9483
- Word: 'fantastic', Ratio: 1.9483
- Word: 'fun', Ratio: 1.9483

--- NEGATIVE ---
- Word: 'waste', Ratio: 3.0796
- Word: 'awful', Ratio: 2.0531
- Word: 'bad', Ratio: 2.0531
- Word: 'boring', Ratio: 2.0531
- Word: 'disliked', Ratio: 2.0531
- Word: 'do', Ratio: 2.0531
- Word: 'dreadful', Ratio: 2.0531
- Word: 'ever', Ratio: 2.0531
- Word: 'hated', Ratio: 2.0531
- Word: 'horrible', Ratio: 2.0531


TODO: Print out the most important words and their corresponding ratios

---
# Does it work? <a name="test"></a>

Let's test our classifier on some examples:

In [11]:

test_examples = [
    "This movie is amazing",
    "I loved this film",
    "Terrible and boring",
    "The worst movie ever",
    "A great and entertaining film",
    "Awful waste of time",
]

print("Testing our classifier:")
print("=" * 60)

for text in test_examples:
    pred, probs = classifier.predict(text, return_probs=True)
    confidence = probs[pred] * 100
    emoji = "correct" if pred == "positive" else "incorrect"
    print(f"{emoji} '{text}'")
    print(f"   --> {pred} ({confidence:.1f}% confident)\n")

Testing our classifier:
correct 'This movie is amazing'
   --> positive (72.7% confident)

correct 'I loved this film'
   --> positive (72.7% confident)

incorrect 'Terrible and boring'
   --> negative (72.2% confident)

incorrect 'The worst movie ever'
   --> negative (75.0% confident)

correct 'A great and entertaining film'
   --> positive (92.9% confident)

incorrect 'Awful waste of time'
   --> negative (94.1% confident)



# How did it do?

Our simple bag-of-words classifier correctly identifies sentiment for straightforward cases.

Even this simple approach works because:
- Positive reviews tend to use words like "amazing", "loved", "great"
- Negative reviews tend to use words like "terrible", "awful", "worst"

Now let's look at some tricky cases!

In [12]:

tricky_examples = [
    # Negation ones
    ("This movie is not good", "negative"),
    ("I don't like this film", "negative"),
    ("Not bad at all", "positive"),
    ("I didn't hate it", "positive"),

    # Say there is some sarcasm or contrasting bits
    ("Oh great, another terrible movie", "negative"),
    ("The acting was good but the story was awful", "mixed"),

    # Say we have words not seen in the data ?
    ("This film is phenomenal", "positive"),
    ("Utterly abysmal", "negative"),
    ("A cinematic triumph", "positive"),

    # Intensity bits
    ("This movie is good", "positive"),
    ("This movie is REALLY good", "positive"),
    ("This movie is soooo good!!!", "positive"),

    # Word variations
    ("I loved it", "positive"),
    ("I'm loving it", "positive"),
    ("Lovable characters", "positive"),
]

print("Testing on TRICKY examples:")
print("=" * 70)

correct = 0
wrong = 0

for text, expected in tricky_examples:
    pred, probs = classifier.predict(text, return_probs=True)
    confidence = probs[pred] * 100

    if expected == "mixed":
        status = "dont know"  # Can't really be right or wrong
    elif pred == expected:
        status = "correct"
        correct += 1
    else:
        status = "incorrect"
        wrong += 1

    print(f"{status} '{text}'")
    print(f"   Expected: {expected}, Got: {pred} ({confidence:.1f}%)\n")

print("=" * 70)
print(f"Results: {correct} correct, {wrong} wrong (excluding 'mixed')")

Testing on TRICKY examples:
correct 'This movie is not good'
   Expected: negative, Got: negative (60.0%)

incorrect 'I don't like this film'
   Expected: negative, Got: positive (57.8%)

incorrect 'Not bad at all'
   Expected: positive, Got: negative (79.1%)

correct 'I didn't hate it'
   Expected: positive, Got: positive (59.1%)

incorrect 'Oh great, another terrible movie'
   Expected: negative, Got: positive (67.3%)

dont know 'The acting was good but the story was awful'
   Expected: mixed, Got: negative (67.2%)

correct 'This film is phenomenal'
   Expected: positive, Got: positive (50.7%)

incorrect 'Utterly abysmal'
   Expected: negative, Got: positive (52.6%)

correct 'A cinematic triumph'
   Expected: positive, Got: positive (61.9%)

correct 'This movie is good'
   Expected: positive, Got: positive (57.8%)

correct 'This movie is REALLY good'
   Expected: positive, Got: positive (57.2%)

correct 'This movie is soooo good!!!'
   Expected: positive, Got: positive (57.8%)

corre

# Advanced TODO:
Can we come up with a negation aware tokenizer? That is -- say you tokenize such that "the movie is not good" is tokenized as ["the", "movie", "is", "not",  "NOT_good"].

In [27]:
# TODO
# 1. Create a new classifier with tokenize_with_negation
# 2. Train it on training_data
# 3. Test on the negation examples from tricky_examples
def tokenize_with_negation(text):
  text = text.lower()
  text = re.sub (r'\b(not|don\'t)\s+(\w+)', r'\1 NOT_\2', text)
  pattern = r'\b\w+\b'
  tokens = re.findall(pattern, text)

  return tokens

classifier_negation = NaiveBayesClassifier(tokenize_with_negation)
classifier_negation.train(training_data)

# Test on negation examples
negation_tests = [
    ("This movie is not good", "negative"),
    ("I don't like this film", "negative"),
    ("Not bad at all", "positive"), # This will fail until we teach the model that not negative means positive
]

print("Testing negation-aware tokenizer:")
for text, expected in negation_tests:
    tokens = tokenize_with_negation(text)
    pred = classifier_negation.predict(text)
    status = "correct" if pred == expected else "incorrect"
    print(f"{status} '{text}'")
    print(f"   Tokens: {tokens}")
    print(f"   Expected: {expected}, Got: {pred}\n")

Trained on 23 documents
Vocabulary size is: 71
Total number of classes: ['positive', 'negative']
Testing negation-aware tokenizer:
correct 'This movie is not good'
   Tokens: ['this', 'movie', 'is', 'not', 'NOT_good']
   Expected: negative, Got: negative

correct 'I don't like this film'
   Tokens: ['i', 'don', 't', 'NOT_like', 'this', 'film']
   Expected: negative, Got: negative

correct 'Not bad at all'
   Tokens: ['not', 'NOT_bad', 'at', 'all']
   Expected: positive, Got: positive



---
# Better Tokenization with BPE <a name="part5"></a>

**Byte-Pair Encoding (BPE)** learns a vocabulary from data that balances:
- Small vocabulary size
- Meaningful units (not just characters)
- No OOV problem (can always fall back to characters)

This is what GPT, BERT, and most modern LLMs use!


Sennrich, Rico, Barry Haddow, and Alexandra Birch. "Neural Machine Translation of Rare Words with Subword Units." Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers). Vol. 1. 2016. http://www.aclweb.org/anthology/P16-1162

Main source: https://github.com/rsennrich/subword-nmt

Below is a simple version of the algorithm.

Two stages: token learner and token segmenter.

Let us first look at token learner, this involves:

*  computing the frequencies of all words in a corpus (we do it synthetically here)
*  start with characters as the basic vocab (characters seen in the corpus)
* to obtain vocabulary of n-merge operations:
    - Obtain most frequent pairs of characters in the corpus
    - add the pair to the list of merges
    - add merged characters to the vocab
* iterate n times

The code below performs this operation.

In [28]:
import collections

def compute_pair_frequencies(vocab):

    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i + 1]] += freq
    return pairs


def merge_vocab(pair, vocab):

    new_vocab = {}
    bigram = re.escape(' '.join(pair))
    pattern = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')

    for word in vocab:
        new_word = pattern.sub(''.join(pair), word)
        new_vocab[new_word] = vocab[word]

    return new_vocab


def train_bpe(corpus_words, num_merges):

    # Initialize: split each word into characters + end marker
    vocab = {}
    for word, freq in corpus_words.items():
        symbols = ' '.join(list(word)) + ' </w>'
        vocab[symbols] = freq

    bpe_codes = {}

    print("BPE Training")
    print("=" * 60)
    print(f"Initial vocabulary: {vocab}\n")

    for i in range(num_merges):
        pairs = compute_pair_frequencies(vocab)

        if not pairs:
            print("No more pairs to merge!")
            break

        # Find most frequent pair
        best_pair = max(pairs, key=pairs.get)
        best_freq = pairs[best_pair]

        # Merge these!
        vocab = merge_vocab(best_pair, vocab)
        bpe_codes[best_pair] = i

        print(f"Merge {i+1}: '{best_pair[0]}' + '{best_pair[1]}' -> '{best_pair[0]+best_pair[1]}' (freq: {best_freq})")

    print(f"\nFinal vocabulary: {vocab}")

    return bpe_codes, vocab

Now we need to train it -- that is, it learns from data!

In [29]:
# Train BPE on a small corpus
corpus_words = {
    'low': 5,
    'lower': 2,
    'newest': 6,
    'widest': 3,
    'new': 4,
}

bpe_codes, final_vocab = train_bpe(corpus_words, num_merges=10)

BPE Training
Initial vocabulary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3, 'n e w </w>': 4}

Merge 1: 'n' + 'e' -> 'ne' (freq: 10)
Merge 2: 'ne' + 'w' -> 'new' (freq: 10)
Merge 3: 'e' + 's' -> 'es' (freq: 9)
Merge 4: 'es' + 't' -> 'est' (freq: 9)
Merge 5: 'est' + '</w>' -> 'est</w>' (freq: 9)
Merge 6: 'l' + 'o' -> 'lo' (freq: 7)
Merge 7: 'lo' + 'w' -> 'low' (freq: 7)
Merge 8: 'new' + 'est</w>' -> 'newest</w>' (freq: 6)
Merge 9: 'low' + '</w>' -> 'low</w>' (freq: 5)
Merge 10: 'new' + '</w>' -> 'new</w>' (freq: 4)

Final vocabulary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3, 'new</w>': 4}


In [30]:
def bpe_encode(word, bpe_codes):

    # Start with characters + end marker
    word = tuple(word) + ('</w>',)

    while len(word) > 1:
        # Find all adjacent pairs
        pairs = [(word[i], word[i+1]) for i in range(len(word)-1)]

        # Find pair with lowest merge rank (earliest learned)
        min_pair = None
        min_rank = float('inf')

        for pair in pairs:
            if pair in bpe_codes and bpe_codes[pair] < min_rank:
                min_pair = pair
                min_rank = bpe_codes[pair]

        # If no pair can be merged, we're done
        if min_pair is None:
            break

        # Merge all occurrences of min_pair
        first, second = min_pair
        new_word = []
        i = 0

        while i < len(word):
            if i < len(word) - 1 and word[i] == first and word[i+1] == second:
                new_word.append(first + second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1

        word = tuple(new_word)

    # Clean up end marker for display
    result = []
    for token in word:
        if token == '</w>':
            continue
        elif token.endswith('</w>'):
            result.append(token[:-4])
        else:
            result.append(token)

    return tuple(result)


# Test BPE encoding
test_words = ['low', 'lower', 'lowest', 'new', 'newer', 'newest', 'widest', 'unknown']

print("\nBPE Encoding Results:")
print("=" * 40)
for word in test_words:
    encoded = bpe_encode(word, bpe_codes)
    print(f"{word:<12} -> {encoded}")


BPE Encoding Results:
low          -> ('low',)
lower        -> ('low', 'e', 'r')
lowest       -> ('low', 'est')
new          -> ('new',)
newer        -> ('new', 'e', 'r')
newest       -> ('newest',)
widest       -> ('w', 'i', 'd', 'est')
unknown      -> ('u', 'n', 'k', 'n', 'o', 'w', 'n')


Okay, observe something interesting (even in this small set of examples). The BPE tokenizer has learned to split the words into:
- `est` (superlative suffix)
- `er` (comparative suffix, eventually)
- `low`, `new` (common roots)

This happened automatically from frequency, not from any linguistic rules!

But, this may not be always true, as you see in the outputs!  

ADVANCED TODO: Train BPE on the full corpus above.

In [31]:
# 1. Prepare the data
# We need to convert our list of sentences (training_data) into a dictionary of word counts
from collections import Counter

full_corpus_counts = Counter()

for text, label in training_data:
    # Use the simple tokenizer we built earlier to get the words
    words = simple_tokenize(text)
    full_corpus_counts.update(words)

print(f"Total unique words in corpus: {len(full_corpus_counts)}")
print(f"Top 5 words: {full_corpus_counts.most_common(5)}")

# 2. Train BPE
# Let's run 50 merges (you can increase this number to learn longer words)
print("\n--- Starting BPE Training ---")
my_bpe_codes, my_bpe_vocab = train_bpe(full_corpus_counts, num_merges=50)

# 3. Test it
print("\n--- Testing BPE Encoding ---")
test_words = ["movie", "movies", "moving", "movement", "wonderful", "wonderfully"]

for word in test_words:
    encoded = bpe_encode(word, my_bpe_codes)
    print(f"{word:<12} -> {encoded}")

Total unique words in corpus: 69
Top 5 words: [('this', 10), ('and', 8), ('movie', 7), ('i', 7), ('of', 5)]

--- Starting BPE Training ---
BPE Training
Initial vocabulary: {'t h i s </w>': 10, 'm o v i e </w>': 7, 'i s </w>': 3, 'a m a z i n g </w>': 1, 'a n d </w>': 8, 'w o n d e r f u l </w>': 1, 'i </w>': 7, 'l o v e d </w>': 1, 'e v e r y </w>': 2, 'm i n u t e </w>': 2, 'o f </w>': 5, 'f i l m </w>': 5, 'b r i l l i a n t </w>': 1, 'a c t i n g </w>': 2, 'g r e a t </w>': 2, 's t o r y </w>': 2, 'o n e </w>': 2, 't h e </w>': 3, 'b e s t </w>': 1, 'm o v i e s </w>': 2, 'h a v e </w>': 1, 's e e n </w>': 1, 'a b s o l u t e l y </w>': 2, 'f a n t a s t i c </w>': 1, 'e n t e r t a i n i n g </w>': 1, 'a </w>': 3, 'b e a u t i f u l </w>': 1, 'm o v i n g </w>': 1, 'e x p e r i e n c e </w>': 2, 'h i g h l y </w>': 1, 'r e c o m m e n d </w>': 1, 'm a s t e r p i e c e </w>': 1, 'e x c e l l e n t </w>': 1, 'w i t h </w>': 2, 's u p e r b </w>': 1, 'p e r f o r m a n c e s </w>': 2